In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets==1.17.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 306 kB 4.3 MB/s 
     |████████████████████████████████| 140 kB 49.6 MB/s 
     |████████████████████████████████| 212 kB 42.9 MB/s 
     |████████████████████████████████| 1.1 MB 34.3 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 144 kB 51.3 MB/s 
     |████████████████████████████████| 271 kB 41.8 MB/s 


In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install pororo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 983 kB 4.3 MB/s 
     |████████████████████████████████| 358 kB 32.3 MB/s 
     |████████████████████████████████| 138 kB 44.2 MB/s 
     |████████████████████████████████| 4.0 MB 40.3 MB/s 
     |████████████████████████████████| 55 kB 4.4 MB/s 
     |████████████████████████████████| 58 kB 6.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://d

In [ ]:
import sys
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm
import os
import re
import time
from collections import Counter

import transformers
from transformers import BertTokenizer,AdamWeightDecay,TFRobertaModel,TFBertModel

import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping,ModelCheckpoint

import sklearn
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import StratifiedKFold

import pororo
from pororo import Pororo

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# Setting results path
data_path='/content/drive/MyDrive/Projects/Data_Debiasing/Korean_Data_Argument/Back_Translation/Data/'

In [ ]:
def chrome_setting():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=chrome_options)
  return driver

In [ ]:
driver=chrome_setting()

In [ ]:
from datasets import load_dataset
dataset = load_dataset('smilegate-ai/kor_unsmile')

In [ ]:
# Crawling
def kor_to_trans(text_data, trans_lang,start_index,final_index):

  target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

  for i in tqdm(range(start_index,final_index)): 
    backtrans = ""
    
    
    if (i!=0)&(i%99==0) and backtrans!="":
      time.sleep(2)
      print('{}th : '.format(i), backtrans)
      np.save(data_path+'kor_to_eng_train_{}_{}.npy'.format(start_index,final_index),trans_list)
    
    try:
      print('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
      driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
      time.sleep(1.5)
      element=WebDriverWait(driver, 10).until(target_present)
      time.sleep(0.1)
      backtrans = element.text 

      if (backtrans=='')|(backtrans==' '):
        element=WebDriverWait(driver, 10).until(target_present)
        backtrans = element.text 
        trans_list.append(backtrans)
      else:
        trans_list.append(backtrans)
    
    except:
      trans_list.append('')

In [ ]:
train = pd.DataFrame({"sentence" : dataset["train"]["문장"]})
print(len(train))

15005


In [ ]:
trans_list=[]
kor_to_trans(train['sentence'], 'en',0,100)
np.save(data_path+'kor_to_eng_train_0_10000.npy',trans_list)

  0%|          | 0/100 [00:00<?, ?it/s]

https://papago.naver.com/?sk=ko&tk=en&st=일안하는 시간은 쉬고싶어서 그런게 아닐까
https://papago.naver.com/?sk=ko&tk=en&st=아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. 10 16까지 IQ 떨어지고 출산 위험은 400% 다.
https://papago.naver.com/?sk=ko&tk=en&st=루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o  doin 진짜 띵곡임 꼭 들어보셈"
https://papago.naver.com/?sk=ko&tk=en&st=홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽에서 고소 가능하냐?
https://papago.naver.com/?sk=ko&tk=en&st=아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런다! 이렇게 2개로 싸우냐.. 내가 볼 땐 둘다 나쁜 말 그 이상도 이하도 아닌데
https://papago.naver.com/?sk=ko&tk=en&st=고향가서 피방가면 동네 부럴 친구들이랑은 뭐 거르는 거 없이 이야기하니까 막 말하게 되더라 당연히 키보드를 치거나 그러지는 않는데 말하는게 많이 거칠어지긴 해 반성해야겠네
https://papago.naver.com/?sk=ko&tk=en&st=예수 십새끼 개새끼 창녀아들 애비실종 가정교육 못받은 무뇌충 허언증 정신병자 사기꾼 망상 근본없는 병신 장애인 짱개 흑인 이슬람 전라도랑 동급
https://papago.naver.com/?sk=ko&tk=en&st=나이쳐먹고 피시방가는 놈들은 대가리에 똥만찬 놈들임
https://papago.naver.com/?sk=ko&tk=en&st=출산청같은 소리하네. 우리나라 계집들 다 의무적으로 군대보내면 해결될 일이다. 정신개조부터 시켜야지. 지할꺼 다하고 무슨 애를 낳고 무슨 자식교육을 하냐?
https://papago.naver.com/?sk=ko&

In [ ]:
trans_list

["I think it's because I want to rest when I don't work",
 '',
 '"Luna\'s solo album was full of Momo. Keep o doin. It\'s a masterpiece. Please listen to it."',
 'There\'s a comment saying, "Send it to Parents\' Union" in Hongpak. If you report it to Parents\' Union, can they file a complaint?',
 "Why are all the comments saying that women are called kimchi girls first? Women do it worse! How can you fight with these two? I don't think they're anything more than bad words",
 "When I go to an internet cafe in my hometown, I talk to my neighborhood friends without skipping anything. Of course, I don't play the keyboard, but it gets rough when I talk. I should reflect on myself",
 'Jesus, the son of a son-in-law, the son of a whore, the son of a father-in-law, the son-in-law, the son-in-law, the son-in-law, the son-in-law, the son-in-law, the son-in-law of a black Muslim Jeolla Province',
 'The ones who go to the PC room after getting old are the ones with a lot of shit on their heads',
 

In [ ]:
eng_data0=np.load(data_path+'kor_to_eng_train_0_10000.npy')

In [ ]:
eng_data=[*eng_data0]
eng_data=pd.DataFrame(eng_data,columns=['eng_sentence'])

In [ ]:
back_train=pd.concat([train[:100],eng_data],axis=1)
back_train

,sentence,eng_sentence
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,I think it's because I want to rest when I don...
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,"""Luna's solo album was full of Momo. Keep o do..."
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,"There's a comment saying, ""Send it to Parents'..."
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,Why are all the comments saying that women are...
...,...,...
95,입갤이 숙대가 높냐 동대가 높냐 숙대에는 꼴펨 많나?,Is there a lot of bad luck in Sookmyung Univer...
96,아니 시벌 터키 그나마 보수적 마인드 때문에 좋아했는데 동성애자들이 저정도로 날뛰는...,"No, I liked Shival Turkey because of its conse..."
97,대구븅신들,Daegu idiots
98,여대를 없애라 여대가 왜 있는지 이해가 안감,Get rid of women's colleges. I don't understan...


In [ ]:
# Reset selenium chrome driver
driver=chrome_setting()

In [ ]:
def kor_to_trans_again(text_data, trans_lang,start_index,final_index):

  target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

  for i in tqdm(range(start_index,final_index)): 
    backtrans = ""
    if (i!=0)&(i%99==0) and backtrans!="" :
      time.sleep(2)
      print('{}th : '.format(i), backtrans)
      np.save(data_path+'kr_title.npy',trans_list)
    
    try:
      driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
      time.sleep(1.5)
      element=WebDriverWait(driver, 10).until(target_present)
      time.sleep(0.1)
      backtrans = element.text 

      if (backtrans=='')|(backtrans==' '):
        element=WebDriverWait(driver, 10).until(target_present)
        backtrans = element.text 
        trans_list.append(backtrans)
      else:
        trans_list.append(backtrans)
    
    except:
      trans_list.append('')

In [ ]:
trans_list=[]
kor_to_trans_again(back_train["eng_sentence"], 'en',0,len(back_train))
np.save(data_path+'kr_title.npy',trans_list)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
kor_data=np.load(data_path+'kr_title.npy')
kor_data=pd.DataFrame(kor_data,columns=['kor_sentence'])
back_train = pd.concat([back_train,kor_data],axis =1)

In [ ]:
back_train

,sentence,eng_sentence,kor_sentence
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,I think it's because I want to rest when I don...,일을 하지 않을 때 쉬고 싶어서 그런 것 같아.
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,,
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,"""Luna's solo album was full of Momo. Keep o do...","""루나의 솔로 앨범은 모모였습니다. 계속 하세요. 그건 걸작이다. 들어주세요."
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,"There's a comment saying, ""Send it to Parents'...","홍박에 ""부모님 연합으로 보내""라는 댓글이 있어요. 어버이연합에 신고하면 고소할 수..."
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,Why are all the comments saying that women are...,왜 댓글에 여자들이 먼저 김치녀라고 하는 거야? 여자는 더 나빠요! 어떻게 이 두 ...
...,...,...,...
95,입갤이 숙대가 높냐 동대가 높냐 숙대에는 꼴펨 많나?,Is there a lot of bad luck in Sookmyung Univer...,숙명대학교나 동대에는 불운이 많은가요?
96,아니 시벌 터키 그나마 보수적 마인드 때문에 좋아했는데 동성애자들이 저정도로 날뛰는...,"No, I liked Shival Turkey because of its conse...","아니, 나는 터키가 보수적인 마인드 때문에 좋아했지만 게이들이 그렇게 열광하는 건 ..."
97,대구븅신들,Daegu idiots,Daegu idiots
98,여대를 없애라 여대가 왜 있는지 이해가 안감,Get rid of women's colleges. I don't understan...,여자 대학을 없애라. 그들이 왜 여기 있는지 모르겠다.
